In [ ]:
import os
import requests
import zipfile
import io
import boto3

# --- Config ---
url = "https://dbw.stat.gov.pl/hvd_new/2479_csv_pl.zip"
bucket_name = os.getenv("MINIO_BUCKET_NAME")
minio_url = os.getenv("MINIO_SERVER_URL")
minio_access_key = os.getenv("MINIO_ROOT_USER")
minio_secret_key = os.getenv("MINIO_ROOT_PASSWORD")

# --- Download ZIP ---
print("⬇️ Downloading data...")
response = requests.get(url)
response.raise_for_status()
print("✅ Downloaded")

# --- Extract CSV files into memory ---
z = zipfile.ZipFile(io.BytesIO(response.content))
file_list = z.namelist()
print("📂 Files in archive:", file_list)

# --- Connect to MinIO ---
s3 = boto3.client(
    "s3",
    endpoint_url=minio_url,
    aws_access_key_id=minio_access_key,
    aws_secret_access_key=minio_secret_key,
)

# --- Upload extracted CSVs to MinIO ---
for fname in file_list:
    with z.open(fname) as f:
        key = f"raw_data/gus_dbw_hvd_2479/{fname}"
        s3.upload_fileobj(f, bucket_name, key)
        print(f"✅ Uploaded {fname} -> s3://{bucket_name}/{key}")

ModuleNotFoundError: No module named 'boto3'